In [34]:
import SBP as sb 
import numpy as np 
import scipy as sp 
import matplotlib.pyplot as plt 
from SBP.legendre import lgl, lagrange


In [6]:
class Element1D:
    """
    1D element for nodal DG / SBP methods.
    Uses Legendre-Gauss-Lobatto (LGL) nodes and maps reference [-1,1] to physical [left,right].
    """
    def __init__(self, index: int, left: float, right: float, n: int):
        self.index = index
        self.left = left
        self.right = right
        self.n = n
        # reference nodes xi and weights (we only need xi)
        xi, _ = lgl(n)
        self.xi = np.array(xi).flatten()
        # physical coordinates
        self.x = ( (self.right - self.left)/2 ) * self.xi + (self.right + self.left)/2
        # Jacobian for mapping
        self.jacobian = (self.right - self.left)/2
        # placeholder for solution values at nodes
        self.solution = np.zeros(n+1)

    def set_solution(self, sol: np.ndarray):
        """Assign solution values at the element's nodes."""
        assert sol.shape == (self.n+1,)
        self.solution = sol.copy()

    def map_to_reference(self, x_phys: float) -> float:
        """Map a physical coordinate back to reference xi in [-1,1]."""
        return (2*x_phys - (self.left + self.right)) / (self.right - self.left)

    def basis_at(self, x_phys: float) -> np.ndarray:
        """Evaluate all Lagrange basis polynomials at a physical point."""
        xi = self.map_to_reference(x_phys)
        return lagrange(self.n, xi)


class Mesh1D:
    """
    1D mesh composed of equally spaced Element1D objects.
    """
    def __init__(self, x_min: float, x_max: float, nex: int, n: int):
        self.x_min = x_min
        self.x_max = x_max
        self.nex = nex
        self.n = n
        self.elements = []
        self.generate_mesh()

    def generate_mesh(self):
        """Partition [x_min,x_max] into nex elements and create Element1D instances."""
        dx = (self.x_max - self.x_min) / self.nex
        for i in range(self.nex):
            left = self.x_min + i*dx
            right = left + dx
            elem = Element1D(i, left, right, self.n)
            self.elements.append(elem)

    def get_element(self, idx: int) -> Element1D:
        """Retrieve element by index."""
        return self.elements[idx]

    def set_solutions(self, U: np.ndarray):
        """Assign solution array U of shape (nex, n+1) to all elements."""
        assert U.shape == (self.nex, self.n+1)
        for i, elem in enumerate(self.elements):
            elem.set_solution(U[i])

    def global_coordinates(self) -> np.ndarray:
        """Return sorted unique global node coordinates."""
        coords = []
        for elem in self.elements:
            coords.extend(elem.x.tolist())
        return np.unique(coords)

    def plot_mesh(self):
        """Simple plot of the mesh nodes and element edges."""
        import matplotlib.pyplot as plt
        X = self.global_coordinates()
        Y = np.zeros_like(X)
        plt.figure(figsize=(8,1))
        plt.plot(X, Y, 'o')
        for elem in self.elements:
            plt.plot(elem.x, np.zeros_like(elem.x), '-')
        plt.yticks([])
        plt.title(f"1D Mesh: {self.nex} elements, degree {self.n}")
        plt.xlabel("x")
        plt.show()

In [10]:
e = Element1D(1, 0, 10, 5)
e.jacobian

5.0

In [35]:
d1 = sb.sbp_d(5)
p1 = sb.sbp_p(5)
d1


array([[-7.50000000e+00,  1.01414159e+01, -4.03618727e+00,
         2.24468465e+00, -1.34991331e+00,  5.00000000e-01],
       [-1.78636495e+00,  1.53765889e-14,  2.52342678e+00,
        -1.15282816e+00,  6.53547507e-01, -2.37781178e-01],
       [ 4.84951048e-01, -1.72125695e+00, -3.38618023e-15,
         1.75296197e+00, -7.86356672e-01,  2.69700611e-01],
       [-2.69700611e-01,  7.86356672e-01, -1.75296197e+00,
         4.77395901e-15,  1.72125695e+00, -4.84951048e-01],
       [ 2.37781178e-01, -6.53547507e-01,  1.15282816e+00,
        -2.52342678e+00, -2.75335310e-14,  1.78636495e+00],
       [-5.00000000e-01,  1.34991331e+00, -2.24468465e+00,
         4.03618727e+00, -1.01414159e+01,  7.50000000e+00]])

In [18]:
#Matrix P
n = 5
out = sb.lgl(n)
roots = np.zeros(n+1)
w = np.zeros(n+1)
w[:] = out[1,:]
roots[:] = out[0,:]
result = np.zeros((n+1,n+1))
l1 = np.zeros(n+1)
for i in range(n+1):
    result = result + (np.outer(sb.lagrange(n,roots[i]),sb.lagrange(n,roots[i])))*w[i]
P = -1*result # The negative one stems from the formulation of the code. The points were sorted from right to left, hence the jacobian is negative. 
print("The matrix P \n", result)


#Matrix Q from P
dq = sb.dlagrange(n)
result1 = np.zeros_like(result)
for i in range(n+1):
    result1[:,i] = sb.lagrange(n,roots[i]) @ np.transpose(dq)* w[i]
result1 = result1  
Q = result1
print("The matrix Q from P \n", Q) 

# Matrix D 
D = sb.dlagrange(n)
print("The matrix D \n", D) 


The matrix P 
 [[0.06666667 0.         0.         0.         0.         0.        ]
 [0.         0.37847496 0.         0.         0.         0.        ]
 [0.         0.         0.55485838 0.         0.         0.        ]
 [0.         0.         0.         0.55485838 0.         0.        ]
 [0.         0.         0.         0.         0.37847496 0.        ]
 [0.         0.         0.         0.         0.         0.06666667]]
The matrix Q from P 
 [[-5.00000000e-01  3.83827195e+00 -2.23951232e+00  1.24548208e+00
  -5.10908383e-01  3.33333333e-02]
 [-1.19090997e-01  5.81965381e-15  1.40014449e+00 -6.39656361e-01
   2.47351364e-01 -1.58520785e-02]
 [ 3.23300699e-02 -6.51452650e-01 -1.87885046e-15  9.72645632e-01
  -2.97616307e-01  1.79800407e-02]
 [-1.79800407e-02  2.97616307e-01 -9.72645632e-01  2.64887115e-15
   6.51452650e-01 -3.23300699e-02]
 [ 1.58520785e-02 -2.47351364e-01  6.39656361e-01 -1.40014449e+00
  -1.04207519e-14  1.19090997e-01]
 [-3.33333333e-02  5.10908383e-01 -1.245482